In [1]:
import requests
from bs4 import BeautifulSoup
import re

states = ['Kansas', 'Nebraska', 'Oklahoma']

state_airports = {}

In [2]:
# get airport codes for states of interest
for st in states:
    print(st)
    temp = []
    r = requests.get("https://www.wunderground.com/history/index.html?error=AMBIGUOUS&query=%s" % st)

    regex = r"\/?history\/?airport\/(\w+)"
    soup = BeautifulSoup(r.text, 'html.parser')

    for i in soup.find_all('a'):
        link = i.get('href')
        if isinstance(link, str):
            matches = re.search(regex, link)
            if matches:
                temp.append(matches.group(1))
    
    state_airports[st] = temp

Kansas
Nebraska
Oklahoma


In [20]:
def non_empty_length(list_):
    counter = 0
    for i in list_:
        if i != '':
            counter+=1
    return counter

def name_list(list_named):
    return (list_named[0], list_named[1:])

def list_to_dict(rows):
    dict_ = {}
    for i in rows:
        if (non_empty_length(i) > 1):
            name, list_ = name_list(i)
            dict_[name] = list_
    return dict_

# get weather for a day at a given airport
def get_airport_weather(airport, year = 2018, day = 29, month = 3):
    link = "https://www.wunderground.com/history/airport/%s/%s/%s/%s/DailyHistory.html" % (airport, year, month, day)
    r = requests.get(link)
    soup = BeautifulSoup(r.text, 'html.parser')
    table = soup.find('table', {'class':'airport-history-summary-table'})
    
    headers = [header.text for header in table.find_all('th') if not header.text == '\xa0']

    rows = []

    for row in table.find_all('tr'):
        temp_row = [val.text.replace('\xa0' , '').strip().replace('\n', ' ') for val in row.find_all('td')]
        if (non_empty_length(temp_row) > 1):
            rows.append(temp_row)
    return pd.DataFrame(rows, columns = ['labels', 'current', 'average', 'record']).set_index('labels')

test = get_airport_weather('KCNU')

In [21]:
test

,current,past,record
labels,,,
Mean Temperature,44°F,51°F,
Max Temperature,52°F,63°F,85°F (1967)
Min Temperature,37°F,40°F,15°F (1944)
Heating Degree Days,20,14,
Month to date heating degree days,492,532,
Since 1 July heating degree days,3838,4104,
Cooling Degree Days,0,0,
Month to date cooling degree days,0,0,
Year to date cooling degree days,0,0,
